## Just a regression

In [9]:
import pandas as pd
from sklearn import linear_model

In [17]:
data_dir = r'/kaggle/input/petfinder-pawpularity-score/'
file_name = r'train.csv'

df_train = pd.read_csv(data_dir + file_name)


In [14]:
file_name = f'test.csv'
df_test = pd.read_csv(data_dir + file_name)

In [10]:
X = df_train[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']]
y = df_train['Pawpularity']

regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [12]:
print(regr.coef_)

[-1.12584593 -2.48009618  2.45013923  0.46844535 -0.3088989   1.21101048
  1.21999797  0.39700325  0.36362689 -0.06627373 -0.71553513 -3.75364608]


In [33]:
pawpularity = regr.predict(df_test[['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
       'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']])

In [36]:
id = df_test.Id.values

In [40]:
data = {'Id': id, 'Pawpularity': pawpularity}

In [41]:
submission = pd.DataFrame(data,columns=['Id','Pawpularity'])

submission.to_csv('submission.csv', index = False)